# OTKA1 post analysis

In [1]:
import pandas as pd
import numpy as np
import mne

/opt/homebrew/Caskroom/miniforge/base/envs/EEGModalNet/lib/python3.11/site-packages/mne/externals/tempita/__init__.py:35: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
  import cgi


In [ ]:
# The change in EEG power in the theta (4 to < 8 Hz), alpha (8 to < 13 Hz), and gamma (> 30 to 42 Hz) bands in the midline frontal (FPz, Fz) and midline occipital (POz, Oz, Iz) regions. 
data = pd.read_excel('data/behavioral_data/PLB_HYP_data_MASTER.xlsx')

In [ ]:
power_data = pd.read_csv('data/classification_datasets/power_sensor.csv', index_col=0)
bands = ['theta', 'alpha', 'gamma']
cols = [i for i in power_data.columns for j in bands if j in i and ('FZ' in i or 'OZ' in i) and 'decibel' not in i]
cols = ['bids_id', 'condition'] + cols

# baseline
baseline = power_data[cols].query("condition.str.contains('baseline1')").reset_index(drop=True).drop('condition', axis=1)
baseline.columns = ['bids_id']+[i + '_baseline' for i in baseline.columns[1:]]
# repeat each row 2 times
baseline = pd.concat([baseline]*2, ignore_index=True).sort_values('bids_id').reset_index(drop=True)

# expeirience
experience = power_data.query("description == 'hypnosis' and condition.str.contains('experience)")
experience = experience[cols].dropna()

In [ ]:
experience = power_data.query("description == 'hypnosis' and condition.str.contains('experience')")
experience = experience[cols].dropna()

In [74]:
experience.join(baseline, on='bids_id', how='inner', lsuffix='_exp', rsuffix='_base')

,bids_id,bids_id_exp,condition,FZ_alpha,FZ_gamma,FZ_theta,OZ_alpha,OZ_gamma,OZ_theta,bids_id_base,FZ_alpha_baseline,FZ_gamma_baseline,FZ_theta_baseline,OZ_alpha_baseline,OZ_gamma_baseline,OZ_theta_baseline
1,1,1,experience1,1.363690,0.058818,0.698948,2.576399,0.049158,0.896892,1,1.789904,0.079990,0.708655,3.997855,0.098021,0.881622
6,1,1,experience3,1.589743,0.051581,0.878828,2.791179,0.335374,1.289431,1,1.789904,0.079990,0.708655,3.997855,0.098021,0.881622
14,2,2,experience2,3.322412,0.180344,1.222884,9.834962,0.139872,1.872974,2,4.103840,0.094774,1.313227,8.679443,0.112204,1.785062
16,2,2,experience3,3.643051,0.113106,1.479129,10.281916,0.078831,2.062579,2,4.103840,0.094774,1.313227,8.679443,0.112204,1.785062
21,3,3,experience1,0.654305,0.127423,1.810013,1.830849,1.553158,1.828773,2,4.103840,0.094774,1.313227,8.679443,0.112204,1.785062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,50,50,experience4,5.699618,0.049928,6.849852,11.349903,0.057757,6.555575,26,9.994670,0.190052,1.533829,47.547006,0.167838,4.395488
506,51,51,experience3,1.065421,0.045619,0.660317,4.457952,0.036258,0.674471,26,9.994670,0.190052,1.533829,47.547006,0.167838,4.395488
508,51,51,experience4,1.286001,0.025697,0.546275,6.088767,0.054217,0.675420,26,9.994670,0.190052,1.533829,47.547006,0.167838,4.395488
514,52,52,experience4,1.550542,0.142301,1.000431,3.802511,0.543600,1.055895,27,6.270510,0.088654,0.726954,9.088744,0.071231,1.000772


In [ ]:
import pickle

with open('docs/1.psd_unaggragated_2nd_analysis.pkl', 'rb') as fp:
    psd1 = pickle.load(fp)

In [ ]:
psd_1 = psd1['01-experience1'].mean(axis=0)
psd_z = (psd_1[3] - np.mean(psd_1[3])) / np.std(psd_1[3])
psd_zc = (psd_1[3] - baseline_1[3].mean()) / np.std(baseline_1[3])
psd_subtracted = psd_1[3] - baseline_1[3].mean()

plt.plot(freqs, psd_z, label='z-transfomed')
plt.plot(freqs, psd_zc, label='z-transfomed with baseline')
plt.plot(freqs, psd_subtracted * 10**12, label='subtracted baseline')
plt.plot(freqs, psd_2, label='decible conversion')
plt.legend()
plt.grid()